In [1]:
import os
import sys
import string
import unicodedata

import numpy as np
import pandas as pd
from tqdm import tqdm

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report

In [ ]:
output_path='test_output85iter_addAlphaDict_2POS.tsv'
model_name = 'crf85iter_AlphaDict_2POS'

In [ ]:
def CRF(x_train, y_train):
    crf = sklearn_crfsuite.CRF(
        algorithm='lbfgs',
        c1=0.1,
        c2=0.1,
        max_iterations=85,
        all_possible_transitions=True
    )
    crf.fit(x_train, y_train)
    labels = list(crf.classes_)
    
    return labels, crf

In [ ]:
# load pretrained word vectors
# get a dict of tokens (key) and their pretrained word vectors (value)
# pretrained word2vec CBOW word vector: https://fgc.stpi.narl.org.tw/activity/videoDetail/4b1141305ddf5522015de5479f4701b1
dim = 0
word_vecs= {}
# open pretrained word vector file
with open('.\\raw_data\\cna.cbow.cwe_p.tar_g.512d.0.txt', encoding='utf-8') as f:
    for line in f:
        tokens = line.strip().split()

        # there 2 integers in the first line: vocabulary_size, word_vector_dim
        if len(tokens) == 2:
            dim = int(tokens[1])
            continue
    
        word = tokens[0]
        vec = np.array([ float(t) for t in tokens[1:] ])
        word_vecs[word] = vec

In [ ]:
print('vocabulary_size: ',len(word_vecs),' word_vector_dim: ',vec.shape)

In [2]:
# simply load data without splitting
def Dataset(data_path):
    with open(data_path, 'r', encoding='utf-8') as f:
        data=f.readlines()#.encode('utf-8').decode('utf-8-sig')
    data_list, data_list_tmp = list(), list()
    article_id_list=list()
    idx=0
    for row in data:
        data_tuple = tuple()
        if row == '\n':
            article_id_list.append(idx)
            idx+=1
            data_list.append(data_list_tmp)
            data_list_tmp = []
        else:
            row = row.strip('\n').split(' ')
            data_tuple = (row[0], row[1])
            data_list_tmp.append(data_tuple)
    if len(data_list_tmp) != 0:
        data_list.append(data_list_tmp)

    return data_list

In [ ]:
# open the pos tag file
df_POSTag = pd.read_csv('.\\processed_data\\train2_POSTag.txt')
df_POSTag.head()

In [ ]:
print(len(df_POSTag))
print(len(df_POSTag['POS'].unique()))

In [41]:
# look up the POSTag txt
# encode the word
def POSTagEncode(data_list, POSTag_csv='.\\processed_data\\train2_POSTag.txt', remove_empty = True):
    
    
    POSTag_list = list()
    LEN_list = list()

    df = pd.read_csv(POSTag_csv)
    if remove_empty:
        df = df[df['entity_text'] != ' ']

    # load overall catagory
    with open('processed_data\\trian2_AND_test_cat.txt', 'r', encoding='utf-8') as f:
        POSTag_label = f.readline()
    POSTag_label = POSTag_label.split(',')

    for idx_list in range(len(data_list)):
        df_temp = df[df['article_id'] == idx_list]
        TXT_temp = list(df_temp['entity_text'])
        POS_temp = list(df_temp['POS'])
        LEN_temp = list(df_temp['LEN'])
        # debug=======
        if len(data_list[idx_list])-len(POS_temp) != 0:
            print('difference '+str(len(data_list[idx_list]))+','+str(len(POS_temp)))
            for idx in range(len(data_list[idx_list])):
                if(data_list[idx_list][idx] != list(df_temp['entity_text'])[idx]):                    
                    if len(data_list[idx_list])>len(POS_temp):
                        print('from '+str(len(data_list[idx_list])))
                        print(type(data_list[idx_list]))
                        data_list[idx_list] = data_list[idx_list][0:idx] + data_list[idx_list][idx+1:len(data_list[idx_list])]
                        break
                    else:
                        print('from '+str(len(POS_temp)))
                        del POS_temp[idx]
                        break
        
        print(idx_list,'\'',str(len(data_list[idx_list])-len(POS_temp)))
        # =========
        POSTag_list_temp = list()
        LEN_list_temp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            
            word_POSTag_temp = list()
            
            # POSTag
            for POSTag_label_code in POSTag_label:
                if POS_temp[idx_tuple] == POSTag_label_code:
                    word_POSTag_temp.append(1)
                else:
                    word_POSTag_temp.append(0)
            POSTag_list_temp.append(word_POSTag_temp)

            # LEN
            LEN_list_temp.append(LEN_temp[idx_tuple])


        POSTag_list.append(POSTag_list_temp)
        LEN_list.append(LEN_list_temp)

    return POSTag_list, LEN_list

In [ ]:
# look up word vectors
# turn each word into its pretrained word vector
# return a list of word vectors corresponding to each token in train.data
def Word2Vector(data_list, embedding_dict):
    
    alphabet_list = list(string.ascii_uppercase)
    embedding_list = list()

    # No Match Word (unknown word) Vector in Embedding
    unk_vector=np.random.rand(*(list(embedding_dict.values())[0].shape))

    for idx_list in range(len(data_list)):
        embedding_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            key = data_list[idx_list][idx_tuple][0] # token

            # alphabet and width
            if unicodedata.normalize('NFKC',key).upper() in alphabet_list:
                key = unicodedata.normalize('NFKC', key).upper()

            if key in embedding_dict:
                value = embedding_dict[key]
            else:
                embedding_dict[key] = unk_vector
                value = embedding_dict[key]
                
            embedding_list_tmp.append(value)
        embedding_list.append(embedding_list_tmp)
    
    return embedding_list

In [ ]:
# input features: pretrained word vectors of each token
# return a list of feature dicts, each feature dict corresponding to each token
def Feature(embed_list, p, l):
    
    with open('processed_data\\trian2_AND_test_cat.txt', 'r', encoding='utf-8') as f:
        POS_unique_list = f.readline()
    POS_unique_list = POS_unique_list.split(',')
    POS_unique_list.append('Start') # 1 if no last word
    
    feature_list = list()
    
    for idx_list in range(len(embed_list)):
        feature_list_tmp = list()
        for idx_tuple in range(len(embed_list[idx_list])):
            
            feature_dict = dict()
            
            # feature of word's POSTag(56+56+1)
            feature_dict['Start'] = 0
            for idx_POS in range(len(POS_unique_list)-1): # exclude Start
                feature_dict[POS_unique_list[idx_POS]] = p[idx_list][idx_tuple][idx_POS]
                # -1 POSTag
                if idx_tuple != 0:
                    feature_dict['last_' + POS_unique_list[idx_POS]] = p[idx_list][idx_tuple-1][idx_POS]
                else:
                    feature_dict['Start'] = 1
                    feature_dict['last_' + POS_unique_list[idx_POS]] = 0
                # +1 POSTag
                if idx_tuple < len(p[idx_list])-1:
                    feature_dict['next_' + POS_unique_list[idx_POS]] = p[idx_list][idx_tuple+1][idx_POS]
                else:
                    feature_dict['Start'] = -1
                    feature_dict['next_' + POS_unique_list[idx_POS]] = 0
                # +2 POSTag
                if idx_tuple < len(p[idx_list])-2:
                    feature_dict['next_' + POS_unique_list[idx_POS]] = p[idx_list][idx_tuple+2][idx_POS]
                else:
                    feature_dict['Start'] = -1
                    feature_dict['next_' + POS_unique_list[idx_POS]] = 0


            # feature of word LEN(1)
            feature_dict['word_LEN'] = l[idx_list][idx_tuple]
            #-----------------
            # feature of word's vector(512)
            for idx_vec in range(len(embed_list[idx_list][idx_tuple])):
                feature_dict['dim_' + str(idx_vec+1)] = embed_list[idx_list][idx_tuple][idx_vec]

            feature_list_tmp.append(feature_dict)
        feature_list.append(feature_list_tmp)
        print(idx_list+1, '\\', len(embed_list)+1, ', # of token:', len(embed_list[idx_list]))
        
    return feature_list

In [ ]:
# get the labels of each tokens in train.data
# return a list of lists of labels
def Preprocess(data_list):
    label_list = list()
    for idx_list in range(len(data_list)):
        label_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            label_list_tmp.append(data_list[idx_list][idx_tuple][1])
        label_list.append(label_list_tmp)
    return label_list

In [ ]:
data_list = Dataset('.\\data\\train2_sample.data')

In [ ]:
p, l = POSTagEncode(data_list)

In [ ]:
p, l = POSTagEncode(data_list)
embed_list = Word2Vector(data_list, word_vecs)
X = Feature(embed_list, p, l)
y = Preprocess(data_list)

In [ ]:
# release resources
if 'embed_list' in globals():
    del embed_list
if 'p' in globals():
    del p
if 'data_list' in globals():
    del data_list

In [ ]:
import joblib

labels, crf = CRF(X, y)
joblib.dump(crf, model_name)

In [5]:
def loadInputFile(file_path):
    with open(file_path, 'r', encoding='utf8') as f:
        dev_set = list()
        
        while(True):
            article_id = f.readline()
            if 'article_id:' not in article_id:
                break
            else:
                dev_set.append(f.readline())
            f.readline()
            f.readline()
            f.readline()
            
    return dev_set

In [19]:
dev_set = loadInputFile('raw_data\\test.txt')

In [ ]:
p, l = POSTagEncode(dev_set, '.\\processed_data\\test_POSTag.txt')
embed_list = Word2Vector(dev_set, word_vecs)
X = Feature(embed_list, p, l)

In [ ]:
y_pred = crf.predict(X)

In [ ]:
print(len(y_pred))
print(len(y_pred[0]))
print(y_pred[0][0])
print(y_pred[0][0][0])
print(len(dev_set[0]))

In [ ]:
i = 0
for dev_id in range(len(y_pred)):
    for pred_id in range(len(y_pred[dev_id])):
        if y_pred[dev_id][pred_id][0] == 'B':
            i = i+1
print(i)

In [ ]:
output="article_id\tstart_position\tend_position\tentity_text\tentity_type\n"
for dev_id in range(len(y_pred)):
    pos=0
    start_pos=None
    end_pos=None
    entity_text=None
    entity_type=None
    for pred_id in range(len(y_pred[dev_id])):
        if y_pred[dev_id][pred_id][0]=='B':
            start_pos=pos
            entity_type=y_pred[dev_id][pred_id][2:]
        elif start_pos is not None and y_pred[dev_id][pred_id][0]=='I'and pred_id<len(y_pred[dev_id])-1 and y_pred[dev_id][pred_id+1][0]=='O':
            end_pos=pos
            entity_text=''.join([dev_set[dev_id][position] for position in range(start_pos,end_pos+1)])
            line=str(dev_id)+'\t'+str(start_pos)+'\t'+str(end_pos+1)+'\t'+entity_text+'\t'+entity_type
            output+=line+'\n'
        pos+=1     

In [ ]:
with open(output_path,'w',encoding='utf-8') as f:
    f.write(output)

In [ ]:
print(output)